# **CellTracksColab**
---

Colab Notebook for Analyzing Migration Tracks generated by [TrackMate](https://imagej.net/plugins/trackmate/)
This Colab notebook is designed to analyze migration tracks, placing emphasis on those generated by using TrackMate, renowned for its proficiency in detailing single-particle tracking data.

Notebook created by [Guillaume Jacquemet](https://cellmig.org/)


# **0. Before getting started**
---

---
<font size = 4>**Important note**

To load your TrackMate outputs, your dataset should be meticulously organized into a two-tiered folder hierarchy as depicted below.

<font size = 4>Here's a common data structure that can work:

## Folder Hierarchy

- 📁 **Experiments** `[Folder_path]`
  - 🌿 **Condition_1** `[‘condition’ is derived from this folder name]`
    - 🔄 **R1** `[‘repeat’ is derived from this folder name]`
      - 📄 `FOV_spots_1.csv`
      - 📄 `FOV_tracks_1.csv`
      - 📄 `FOV_spots_2.csv`
      - 📄 `FOV_tracks_2.csv`
    - 🔄 **R2**
      - 📄 `FOV_spots_1.csv`
      - 📄 `FOV_tracks_1.csv`
      - 📄 `FOV_spots_2.csv`
      - 📄 `FOV_tracks_2.csv`
  - 🌿 **Condition_2**
    - 🔄 **R1**
    - 🔄 **R2**

<font size = 4>In this representation, different symbols are used to represent folders and files clearly:

📁 represents the main folder or directory.
🌿 represents the condition folders.
🔄 represents the repeat folders.
📄 represents the individual CSV files.

---
<font size = 4>**Important note 2**

Be advised of one significant limitation inherent to this notebook.

<font size = 4 color="red">**Part2 does not support Track splitting**</font>. For users aiming to compute additional track metrics within this environment, it is crucial to disable track splitting in TrackMate.

It’s important to clarify that the absence of track splitting support does not hinder the notebook's ability to compile and display results in part 3 of the analysis process.







In [ ]:
# @title #MIT License

print("""
**MIT License**

Copyright (c) 2023 Guillaume Jacquemet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.""")

#Version log
---
While I strive to provide accurate and helpful information, please be aware that:
  - This notebook may contain bugs.
  - Features are currently limited and will be expanded in future releases.

We encourage users to report any issues or suggestions for improvement. Please check the [repository](https://github.com/guijacquemet/CellTracksColab) regularly for updates and the latest version of this notebook.

#### Known Issues:
- Part 4 is limited and unstable.


<font size = 4>**Version 0.2**
  - Added support for 3D tracks
  - New documentation and metrics added.

<font size = 4>**Version 0.1**
This is the first release of this notebook.

---

--------------------------------------------------------
# **Part 1: Prepare the session and Load your data**
--------------------------------------------------------


## **1.1. Install key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Play to install
!pip -q install pandas scikit-learn
!pip -q install hdbscan
!pip -q install umap-learn
!pip -q install plotly


import requests

# URL to the raw content of the version file in the repository
version_url = "https://raw.githubusercontent.com/guijacquemet/CellTracksColab/main/Notebook/latest_version.txt"

# Current version of the notebook the user is running
current_version = "0.2"

try:
    response = requests.get(version_url)
    response.raise_for_status()  # Check whether the request was successful
    latest_version = response.text.strip()  # Get the latest version from the version file

    if latest_version != current_version:
        print(f"A newer version of this notebook is available: {latest_version}. "
              f"Please download the latest version from the repository.")
    else:
        print("You are running the latest version of this notebook.")
except requests.RequestException as e:
    print("Could not check for the latest version of the notebook.")


## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive



## **1.3. Load your (or test) data**

<font size = 4> Please ensure that your data is properly organised (see above)


In [ ]:
#@markdown ###Provide the path to your dataset

Folder_path = ''  # @param {type: "string"}

#@markdown ###Or use a test dataset
Use_test_dataset = True #@param {type:"boolean"}

#@markdown ###Provide the path to your Result folder

Results_Folder = "/content"  # @param {type: "string"}

import os
import re
import glob
import pandas as pd

def populate_columns(df, filepath):
    # Extract the parts of the file path
    path_parts = os.path.normpath(filepath).split(os.sep)

    if len(path_parts) < 3:
        # if there are not enough parts in the path to extract folder and parent folder
        print(f"Error: Cannot extract parent folder and folder from the filepath: {filepath}")
        return df

    # Assuming that the file is located at least two levels deep in the directory structure
    folder_name = path_parts[-2]  # The folder name is the second last part of the path
    parent_folder_name = path_parts[-3]  # The parent folder name is the third last part of the path

    df['File_name'] = os.path.splitext(os.path.basename(filepath))[0]
    df['Condition'] = parent_folder_name  # Populate 'Condition' with the parent folder name
    df['experiment_nb'] = folder_name  # Populate 'Repeat' with the folder name

    return df

def load_and_populate(file_pattern, usecols=None):
    df_list = []
    pattern = re.compile(file_pattern)  # Compile the file pattern to a regex object

    # Go through each root, dirs, files triplet returned by os.walk
    for dirpath, dirnames, filenames in os.walk(Folder_path):
        for filename in filenames:
            if pattern.match(filename):  # Check if the filename matches the file pattern
                filepath = os.path.join(dirpath, filename)
                df = pd.read_csv(filepath, skiprows=[1, 2, 3], usecols=usecols)
                df_list.append(populate_columns(df, filepath))

    if not df_list:  # if df_list is empty, return an empty DataFrame
        print(f"No files found with pattern: {file_pattern}")
        return pd.DataFrame()

    merged_df = pd.concat(df_list, ignore_index=True)
    return merged_df

def sort_and_generate_repeat(merged_df):
    merged_df.sort_values(['Condition', 'experiment_nb'], inplace=True)
    merged_df = merged_df.groupby('Condition', group_keys=False).apply(generate_repeat)
    return merged_df

def generate_repeat(group):
    unique_experiment_nbs = sorted(group['experiment_nb'].unique())
    experiment_nb_to_repeat = {experiment_nb: i+1 for i, experiment_nb in enumerate(unique_experiment_nbs)}
    group['Repeat'] = group['experiment_nb'].map(experiment_nb_to_repeat)
    return group

if (Use_test_dataset):
  print("Downloading test dataset")
  !wget -nc -O /content/T_cell_dataset.zip https://github.com/guijacquemet/CellTracksColab/raw/main/Test_dataset/T_cell_dataset.zip && unzip -q /content/T_cell_dataset.zip -d /content
  Folder_path = "/content/Tracks"

print("Merging CSV files....")

merged_tracks_df = load_and_populate(r'.*tracks.*\.csv')  # Use raw string to avoid escape character issues
merged_tracks_df = sort_and_generate_repeat(merged_tracks_df)
merged_tracks_df['Unique_ID'] = merged_tracks_df['Condition'] + "_" + merged_tracks_df['experiment_nb'] + "_" + merged_tracks_df['TRACK_ID'].astype(str)
merged_tracks_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)

merged_spots_df = load_and_populate(r'.*spots.*\.csv')  # Use raw string to avoid escape character issues
merged_spots_df = sort_and_generate_repeat(merged_spots_df)
merged_spots_df['Unique_ID'] = merged_spots_df['Condition'] + "_" + merged_spots_df['experiment_nb'] + "_" + merged_spots_df['TRACK_ID'].astype(str)
merged_spots_df.to_csv(Results_Folder + '/' + 'merged_Spots.csv', index=False, compression='gzip')

print("Done")


## **1.4. Visualise your tracks**

In [ ]:
# @title ##Run the cell and choose the file you want to inspect


import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

# Extract unique filenames from the dataframe
filenames = merged_spots_df['File_name'].unique()

# Create a Dropdown widget with the filenames
filename_dropdown = widgets.Dropdown(
    options=filenames,
    value=filenames[0] if len(filenames) > 0 else None,  # Default selected value
    description='File Name:',
)

def plot_coordinates(filename):
    if filename:
        # Filter the DataFrame based on the selected filename
        filtered_df = merged_spots_df[merged_spots_df['File_name'] == filename]

        plt.figure(figsize=(10, 8))
        for unique_id in filtered_df['Unique_ID'].unique():
            unique_df = filtered_df[filtered_df['Unique_ID'] == unique_id].sort_values(by='POSITION_T')
            plt.plot(unique_df['POSITION_X'], unique_df['POSITION_Y'], marker='o', linestyle='-', markersize=2)

        plt.xlabel('POSITION_X')
        plt.ylabel('POSITION_Y')
        plt.title(f'Coordinates for {filename}')
        plt.show()
    else:
        print("No valid filename selected")

# Link the Dropdown widget to the plotting function
interact(plot_coordinates, filename=filename_dropdown)


--------------------------------------------------------
# **Part 2: Compute additional metrics**
--------------------------------------------------------
<font size = 4 color="red">Part2 does not support Track splitting</font>.
<font size = 4> For users aiming to compute additional track metrics within this environment, it is crucial to disable track splitting in TrackMate.

## **Directionality**
To calculate the directionality of a track in 3D space, we consider a series of points each with \(x\), \(y\), and \(z\) coordinates, sorted by time. The directionality, denoted as \(D\), is calculated using the formula:

$$ D = \frac{d_{\text{euclidean}}}{d_{\text{total path}}} $$

where \($d_{\text{euclidean}}$\) is the Euclidean distance between the first and the last points of the track, calculated as:

$$ d_{\text{euclidean}} = \sqrt{(x_{\text{end}} - x_{\text{start}})^2 + (y_{\text{end}} - y_{\text{start}})^2 + (z_{\text{end}} - z_{\text{start}})^2} $$

and \($d_{\text{total path}}$\) is the sum of the Euclidean distances between all consecutive points in the track, representing the total path length traveled. If the total path length is zero, the directionality is defined to be zero. This measure provides insight into the straightness of the path taken, with a value of 1 indicating a straight path between the start and end points, and values approaching 0 indicating more circuitous paths.


In [ ]:
# @title ##Calculate directionality
import pandas as pd
import numpy as np

# Function to calculate Directionality
def calculate_directionality(group):
    group = group.sort_values('POSITION_T')
    start_point = group.iloc[0][['POSITION_X', 'POSITION_Y', 'POSITION_Z']]
    end_point = group.iloc[-1][['POSITION_X', 'POSITION_Y', 'POSITION_Z']]

    # Calculating Euclidean distance in 3D between start and end points
    euclidean_distance = np.sqrt((end_point - start_point).pow(2).sum())

    # Calculating the total path length in 3D
    deltas = np.sqrt(group['POSITION_X'].diff().fillna(0)**2 +
                     group['POSITION_Y'].diff().fillna(0)**2 +
                     group['POSITION_Z'].diff().fillna(0)**2)
    total_path_length = deltas.sum()

    # Calculating Directionality
    D = euclidean_distance / total_path_length if total_path_length != 0 else 0

    return pd.Series({'Directionality': D})

# Sort the DataFrame by 'Unique_ID' and 'POSITION_T'
merged_spots_df.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate directionality for each track
df_directionality = merged_spots_df.groupby('Unique_ID').apply(calculate_directionality).reset_index()

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_directionality.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the directionality back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_directionality, on='Unique_ID', how='left')

merged_tracks_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)

In [ ]:
import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# @title ##Plot Directionality


# List of variables to plot
variables_to_plot = ["Directionality"]

# Initialize PDF
pdf_pages = PdfPages(Results_Folder+'Boxplots.pdf')

number_of_unique_conditions = merged_tracks_df['Condition'].nunique()  # Get the number of unique conditions
width_per_condition = 3  # Width in inches per condition

fig, axes = plt.subplots(len(variables_to_plot), 1, figsize=(number_of_unique_conditions * width_per_condition, 4))


# Make sure axes is a list in case there's only one subplot
if len(variables_to_plot) == 1:
    axes = [axes]

for ax, var in zip(axes, variables_to_plot):
    # Extract the data for this variable
    data_for_var = merged_tracks_df[['Condition', var]]

        # Calculate the Interquartile Range (IQR) to identify outliers
    Q1 = merged_tracks_df[var].quantile(0.25)
    Q3 = merged_tracks_df[var].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the outliers
    lower_bound = Q1 - 8 * IQR
    upper_bound = Q3 + 8 * IQR

    # Save this data to a CSV file
    data_for_var.to_csv(f"{Results_Folder}/data_for_{var}.csv", index=False)
    sns.boxplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, color='lightgray')  # Boxplot
    sns.stripplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
    ax.set_ylim([max(min(merged_tracks_df[var]), lower_bound), min(max(merged_tracks_df[var]), upper_bound)])

    ax.set_title(f"{var}")
    ax.set_xlabel('Condition')
    ax.set_ylabel(var)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the layout to prevent overlap
plt.show()
# Save the figure to a PDF
pdf_pages.savefig(fig)

# Close the PDF
pdf_pages.close()

## **Tortuosity**
This measure provides insight into the curvature and complexity of the path taken, with a value of 1 indicating a straight path between the start and end points, and values greater than 1 indicating paths with more twists and turns.
To calculate the tortuosity of a track in 3D space, we consider a series of points each with \(x\), \(y\), and \(z\) coordinates, sorted by time. The tortuosity, denoted as \(T\), is calculated using the formula:

$$ T = \frac{d_{\text{total path}}}{d_{\text{euclidean}}} $$



In [ ]:
import pandas as pd
import numpy as np

# @title ##Calculate tortuosity

def calculate_tortuosity(group):
    group = group.sort_values('POSITION_T')
    start_point = group.iloc[0][['POSITION_X', 'POSITION_Y', 'POSITION_Z']]
    end_point = group.iloc[-1][['POSITION_X', 'POSITION_Y', 'POSITION_Z']]

    # Calculating Euclidean distance in 3D between start and end points
    euclidean_distance = np.sqrt((end_point - start_point).pow(2).sum())

    # Calculating the total path length in 3D
    deltas = np.sqrt(group['POSITION_X'].diff().fillna(0)**2 +
                     group['POSITION_Y'].diff().fillna(0)**2 +
                     group['POSITION_Z'].diff().fillna(0)**2)
    total_path_length = deltas.sum()

    # Calculating Tortuosity
    T = total_path_length / euclidean_distance if euclidean_distance != 0 else 0

    return pd.Series({'Tortuosity': T})

# Assuming merged_spots_df is your DataFrame
# Sort the DataFrame by 'Unique_ID' and 'POSITION_T'
merged_spots_df.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate tortuosity for each track
df_tortuosity = merged_spots_df.groupby('Unique_ID').apply(calculate_tortuosity).reset_index()

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_tortuosity.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the tortuosity back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_tortuosity, on='Unique_ID', how='left')

# Save the DataFrame with the calculated tortuosity
merged_tracks_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)


In [ ]:
# @title ##Plot Tortuosity

import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# List of variables to plot
variables_to_plot = ["Tortuosity"]

# Initialize PDF
pdf_pages = PdfPages(Results_Folder+'Boxplots.pdf')

number_of_unique_conditions = merged_tracks_df['Condition'].nunique()  # Get the number of unique conditions
width_per_condition = 3  # Width in inches per condition

fig, axes = plt.subplots(len(variables_to_plot), 1, figsize=(number_of_unique_conditions * width_per_condition, 4))


# Make sure axes is a list in case there's only one subplot
if len(variables_to_plot) == 1:
    axes = [axes]

for ax, var in zip(axes, variables_to_plot):
    # Extract the data for this variable
    data_for_var = merged_tracks_df[['Condition', var]]

        # Calculate the Interquartile Range (IQR) to identify outliers
    Q1 = merged_tracks_df[var].quantile(0.25)
    Q3 = merged_tracks_df[var].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the outliers
    lower_bound = Q1 - 5 * IQR
    upper_bound = Q3 + 5 * IQR

    # Save this data to a CSV file
    data_for_var.to_csv(f"{Results_Folder}/data_for_{var}.csv", index=False)
    sns.boxplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, color='lightgray')  # Boxplot
    sns.stripplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
    ax.set_ylim([max(min(merged_tracks_df[var]), lower_bound), min(max(merged_tracks_df[var]), upper_bound)])

    ax.set_title(f"{var}")
    ax.set_xlabel('Condition')
    ax.set_ylabel(var)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the layout to prevent overlap
plt.show()
# Save the figure to a PDF
pdf_pages.savefig(fig)

# Close the PDF
pdf_pages.close()

## **Calculate the total turning angle**

This measure provides insight into the cumulative amount of turning along the path, with a value of 0 indicating a straight path with no turning, and higher values indicating paths with more turning.

To calculate the Total Turning Angle of a track in 3D space, we consider a series of points each with \(x\), \(y\), and \(z\) coordinates, sorted by time. The Total Turning Angle, denoted as \(A\), is the sum of the angles between each pair of consecutive direction vectors along the track, representing the cumulative amount of turning along the path.

For each pair of consecutive segments in the track, we calculate the direction vectors \( $\vec{v_1}$ \) and \($ \vec{v_2}$ \), and the angle \($ \theta$ \) between them is calculated using the formula:

$$ \cos(\theta) = \frac{\vec{v_1} \cdot \vec{v_2}}{||\vec{v_1}|| \cdot ||\vec{v_2}||} $$

where \( $\vec{v_1} \cdot$ $\vec{v_2}$ \) is the dot product of the direction vectors, and \( $||\vec{v_1}||$ \) and \( $||\vec{v_2}||$ \) are the magnitudes of the direction vectors. The Total Turning Angle \( $A$ \) is then the sum of all the angles \( \$theta$ \) calculated between each pair of consecutive direction vectors along the track:

$$ A = \sum \theta $$

If either of the direction vectors is a zero vector, the angle between them is undefined, and such cases are skipped in the calculation.


In [ ]:
# @title ##Calculate the total turning angle

import pandas as pd
import numpy as np

def calculate_total_turning_angle(group):
    group = group.sort_values('POSITION_T')
    directions = group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].diff().dropna()
    total_turning_angle = 0

    for i in range(1, len(directions)):
        dir1 = directions.iloc[i - 1]
        dir2 = directions.iloc[i]

        if np.linalg.norm(dir1) == 0 or np.linalg.norm(dir2) == 0:
            continue

        cos_angle = np.dot(dir1, dir2) / (np.linalg.norm(dir1) * np.linalg.norm(dir2))
        cos_angle = np.clip(cos_angle, -1, 1)
        angle = np.degrees(np.arccos(cos_angle))
        total_turning_angle += angle

    return pd.Series({'Total_Turning_Angle': total_turning_angle})

# Assuming merged_spots_df is your DataFrame
merged_spots_df.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

df_turning_angle = merged_spots_df.groupby('Unique_ID').apply(calculate_total_turning_angle).reset_index()

# Check if 'Total_Turning_Angle' is in the columns of df_turning_angle
if 'Total_Turning_Angle' not in df_turning_angle.columns:
    print("Error: 'Total_Turning_Angle' not in df_turning_angle columns")

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_turning_angle.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the total turning angle back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_turning_angle, on='Unique_ID', how='left')

# Save the DataFrame with the calculated total turning angle
merged_tracks_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)



In [ ]:
# @title ##Plot Total Turning Angle

import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# List of variables to plot
variables_to_plot = ["Total_Turning_Angle"]

# Initialize PDF
pdf_pages = PdfPages(Results_Folder+'Boxplots.pdf')

number_of_unique_conditions = merged_tracks_df['Condition'].nunique()  # Get the number of unique conditions
width_per_condition = 3  # Width in inches per condition

fig, axes = plt.subplots(len(variables_to_plot), 1, figsize=(number_of_unique_conditions * width_per_condition, 4))


# Make sure axes is a list in case there's only one subplot
if len(variables_to_plot) == 1:
    axes = [axes]

for ax, var in zip(axes, variables_to_plot):
    # Extract the data for this variable
    data_for_var = merged_tracks_df[['Condition', var]]

        # Calculate the Interquartile Range (IQR) to identify outliers
    Q1 = merged_tracks_df[var].quantile(0.25)
    Q3 = merged_tracks_df[var].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the outliers
    lower_bound = Q1 - 8 * IQR
    upper_bound = Q3 + 8 * IQR

    # Save this data to a CSV file
    data_for_var.to_csv(f"{Results_Folder}/data_for_{var}.csv", index=False)
    sns.boxplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, color='lightgray')  # Boxplot
    sns.stripplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
    ax.set_ylim([max(min(merged_tracks_df[var]), lower_bound), min(max(merged_tracks_df[var]), upper_bound)])

    ax.set_title(f"{var}")
    ax.set_xlabel('Condition')
    ax.set_ylabel(var)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the layout to prevent overlap
plt.show()
# Save the figure to a PDF
pdf_pages.savefig(fig)

# Close the PDF
pdf_pages.close()

## **Calculate the Spatial Coverage**

Spatial coverage provides insight into the spatial extent covered by the object's movement, with higher values indicating that the object has covered a larger area or volume during its movement.


To calculate the spatial coverage of a track in 2D or 3D space, we consider a series of points each with \(x\), \(y\), and optionally \(z\) coordinates, sorted by time. The spatial coverage, denoted as \(S\), represents the area (in 2D) or volume (in 3D) enclosed by the convex hull formed by the points in the track. It provides insight into the spatial extent covered by the moving object.

#### In the implementation below we:
1. **Check Dimensionality**:
   - If the variance of the \(z\) coordinates is zero, implying all \(z\) coordinates are the same, the spatial coverage is calculated in 2D using only the \(x\) and \(y\) coordinates.
   - If the \(z\) coordinates vary, the spatial coverage is calculated in 3D using the \(x\), \(y\), and \(z\) coordinates.

2. **Form Convex Hull**:
   - In 2D, a minimum of 3 non-collinear points is required to form a convex hull.
   - In 3D, a minimum of 4 non-coplanar points is required to form a convex hull.
   - If the required minimum points are not available, the spatial coverage is defined to be zero.

3. **Calculate Spatial Coverage**:
   - In 2D, the spatial coverage \(S\) is the area of the convex hull formed by the points in the track.
   - In 3D, the spatial coverage \(S\) is the volume of the convex hull formed by the points in the track.

#### Formula:
- For 2D Spatial Coverage (Area of Convex Hull), if points are \(P_1(x_1, y_1), P_2(x_2, y_2), \ldots, P_n(x_n, y_n)\):
  $$ S_{2D} = \text{Area of Convex Hull formed by } P_1, P_2, \ldots, P_n $$

- For 3D Spatial Coverage (Volume of Convex Hull), if points are \(P_1(x_1, y_1, z_1), P_2(x_2, y_2, z_2), \ldots, P_n(x_n, y_n, z_n)\):
  $$ S_{3D} = \text{Volume of Convex Hull formed by } P_1, P_2, \ldots, P_n $$



In [ ]:
# @title ##Calculate the Spatial Coverage

import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull

def calculate_spatial_coverage(group):
    group = group.sort_values('POSITION_T')
    coords = group[['POSITION_X', 'POSITION_Y', 'POSITION_Z']].values

    # Check the variance of Z coordinates
    z_variance = np.var(coords[:, 2])

    if z_variance == 0:  # If variance of Z is 0, calculate 2D spatial coverage
        if len(coords) < 3:  # Need at least 3 points for a 2D convex hull
            return pd.Series({'Spatial_Coverage': 0})

        try:
            coords_2d = coords[:, :2]  # Use only X and Y coordinates
            hull_2d = ConvexHull(coords_2d, qhull_options='QJ')  # 'QJ' joggles the input to avoid precision errors
            spatial_coverage = hull_2d.volume  # Area of the convex hull in 2D
        except Exception as e:
            print(f"Error calculating 2D spatial coverage: {e}")
            spatial_coverage = 0
    else:  # If variance of Z is not 0, calculate 3D spatial coverage
        if len(coords) < 4:  # Need at least 4 points for a 3D convex hull
            return pd.Series({'Spatial_Coverage': 0})

        try:
            hull = ConvexHull(coords, qhull_options='QJ')  # 'QJ' joggles the input to avoid precision errors
            spatial_coverage = hull.volume  # Volume of the convex hull in 3D
        except Exception as e:
            print(f"Error calculating 3D spatial coverage: {e}")
            spatial_coverage = 0

    return pd.Series({'Spatial_Coverage': spatial_coverage})

# Assuming merged_spots_df is your DataFrame
merged_spots_df.sort_values(by=['Unique_ID', 'POSITION_T'], inplace=True)

# Calculate spatial coverage for each track
df_spatial_coverage = merged_spots_df.groupby('Unique_ID').apply(calculate_spatial_coverage).reset_index()

# Find the overlapping columns between the two DataFrames, excluding the merging key
overlapping_columns = merged_tracks_df.columns.intersection(df_spatial_coverage.columns).drop('Unique_ID')

# Drop the overlapping columns from the left DataFrame
merged_tracks_df.drop(columns=overlapping_columns, inplace=True)

# Merge the spatial coverage back into the original DataFrame
merged_tracks_df = pd.merge(merged_tracks_df, df_spatial_coverage, on='Unique_ID', how='left')

# Save the DataFrame with the calculated spatial coverage
merged_tracks_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)



In [ ]:
# @title ##Plot The Spatial Coverage

import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# List of variables to plot
variables_to_plot = ["Spatial_Coverage"]

# Initialize PDF
pdf_pages = PdfPages(Results_Folder+'Boxplots.pdf')

number_of_unique_conditions = merged_tracks_df['Condition'].nunique()  # Get the number of unique conditions
width_per_condition = 3  # Width in inches per condition

fig, axes = plt.subplots(len(variables_to_plot), 1, figsize=(number_of_unique_conditions * width_per_condition, 4))


# Make sure axes is a list in case there's only one subplot
if len(variables_to_plot) == 1:
    axes = [axes]

for ax, var in zip(axes, variables_to_plot):
    # Extract the data for this variable
    data_for_var = merged_tracks_df[['Condition', var]]

        # Calculate the Interquartile Range (IQR) to identify outliers
    Q1 = merged_tracks_df[var].quantile(0.25)
    Q3 = merged_tracks_df[var].quantile(0.75)
    IQR = Q3 - Q1

    # Define bounds for the outliers
    lower_bound = Q1 - 8 * IQR
    upper_bound = Q3 + 8 * IQR

    # Save this data to a CSV file
    data_for_var.to_csv(f"{Results_Folder}/data_for_{var}.csv", index=False)
    sns.boxplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, color='lightgray')  # Boxplot
    sns.stripplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, hue='Repeat', dodge=True, jitter=True, alpha=0.2)  # Individual data points
    ax.set_ylim([max(min(merged_tracks_df[var]), lower_bound), min(max(merged_tracks_df[var]), upper_bound)])

    ax.set_title(f"{var}")
    ax.set_xlabel('Condition')
    ax.set_ylabel(var)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

plt.tight_layout(rect=[0, 0, 0.85, 1])  # Adjust the layout to prevent overlap
plt.show()
# Save the figure to a PDF
pdf_pages.savefig(fig)

# Close the PDF
pdf_pages.close()

-------------------------------------------

# **Part 3: Plot track parameters**
-------------------------------------------

<font size = 4> In this section you can plot all the track parameters previously computed. Data and graphs are automatically saved in your result folder.


In [ ]:
# @title ##Plot track parameters

import ipywidgets as widgets
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Create a list of potential variables from DataFrame columns
all_columns = merged_tracks_df.columns.tolist()

# Remove unwanted columns like 'condition' and 'repeat' from the list
selectable_columns = [col for col in all_columns if col not in ['Condition', 'experiment_nb', 'File_name', 'Repeat', 'Unique_ID', 'LABEL', 'TRACK_INDEX', 'TRACK_ID', 'TRACK_X_LOCATION', 'TRACK_Y_LOCATION', 'TRACK_Z_LOCATION']]

# Create checkboxes for selectable columns
variable_checkboxes = [widgets.Checkbox(value=False, description=col) for col in selectable_columns]

# Arrange and display checkboxes in the notebook
display(widgets.VBox([
    widgets.Label('Variables to Plot:'),
    widgets.GridBox(variable_checkboxes, layout=widgets.Layout(grid_template_columns="repeat(%d, 300px)" % 3)),
]))

# Define the plotting function
def plot_selected_vars(button):
  print("Plotting in progress...")

  variables_to_plot = [box.description for box in variable_checkboxes if box.value]

  pdf_pages = PdfPages(f"{Results_Folder}/boxplots.pdf")

# Determine the number of variables to plot
  n_plots = len(variables_to_plot)

# If no variables are selected, avoid creating a plot
  if n_plots == 0:
    print("No variables selected for plotting")
  else:
    # Set the height of each subplot and figure width
    subplot_height = 4  # Adjust as per your requirement
    number_of_unique_conditions = merged_tracks_df['Condition'].nunique()  # Get the number of unique conditions
    width_per_condition = 3  # Width in inches per condition

    fig_width = number_of_unique_conditions * width_per_condition  # Adjust as per your requirement

    # Calculate the total figure height
    fig_height = n_plots * subplot_height

    # Create subplots with dynamic figure size
    fig, axes = plt.subplots(n_plots, 1, figsize=(fig_width, fig_height))

    # Make axes iterable in case there's only one subplot
    if n_plots == 1:
        axes = [axes]

    for ax, var in zip(axes, variables_to_plot):
        data_for_var = merged_tracks_df[['Condition', var]]

        # Calculate the Interquartile Range (IQR) to identify outliers
        Q1 = merged_tracks_df[var].quantile(0.25)
        Q3 = merged_tracks_df[var].quantile(0.75)
        IQR = Q3 - Q1

    # Define bounds for the outliers
        lower_bound = Q1 - 8 * IQR
        upper_bound = Q3 + 8 * IQR

        data_for_var.to_csv(f"{Results_Folder}/data_for_{var}.csv", index=False)
        sns.boxplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, color='lightgray')
        sns.stripplot(x='Condition', y=var, data=merged_tracks_df, ax=ax, hue='Repeat', dodge=True, jitter=True, alpha=0.2)
        ax.set_ylim([max(min(merged_tracks_df[var]), lower_bound), min(max(merged_tracks_df[var]), upper_bound)])

        ax.set_title(f"{var}")
        ax.set_xlabel('Condition')
        ax.set_ylabel(var)
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), title='Repeat')

    plt.tight_layout(rect=[0, 0, 0.85, 1])
    pdf_pages.savefig(fig)
    pdf_pages.close()
    plt.show()

# Create a button that will execute the plotting function when clicked
button = widgets.Button(description="Plot Selected Variables", layout=widgets.Layout(width='400px'))
button.on_click(plot_selected_vars)
display(button)


--------
# **Part 4: Explore your high-dimensional data (work in progress)**
--------

<font size = 4> The workflow provided below is inspired by [CellPlato](https://github.com/Michael-shannon/cellPLATO)

### **4.1: Choose the track metrics to use for clustering**


In [ ]:
# @title ##Choose the track metrics to use

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

selected_df = pd.DataFrame()

# Filter out non-numeric columns
numeric_df = merged_tracks_df.select_dtypes(include=['float64', 'int64'])  # Selecting only numeric columns

# Create a list of column names
column_names = numeric_df.columns.tolist()

# Create a checkbox for each column
checkboxes = [widgets.Checkbox(value=True, description=col, indent=False) for col in column_names]

# Arrange checkboxes in a 2x grid
grid = widgets.GridBox(checkboxes, layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))

# Create a button to trigger the selection
button = widgets.Button(description="Select the track parameters", layout=widgets.Layout(width='400px'))

# Define the button click event handler
def on_button_click(b):
    global selected_df  # Declare selected_df as global

    # Get the selected columns from the checkboxes
    selected_columns = [box.description for box in checkboxes if box.value]

    # Extract the selected columns from the DataFrame
    selected_df = numeric_df[selected_columns]

    # Check if the DataFrame has any NaN values and print a warning if it does.
    nan_columns = selected_df.columns[selected_df.isna().any()].tolist()

    if nan_columns:
        warnings.warn(f"The DataFrame contains NaN values in the following columns: {', '.join(nan_columns)}")
        for col in nan_columns:
            selected_df = selected_df.dropna(subset=[col])  # Drop NaN values only from columns containing them

    print("Done")

# Set the button click event handler
button.on_click(on_button_click)

# Display the grid of checkboxes and the button
display(grid, button)


### **4.2: UMAP and HDBSCAN**

<font size = 4> The given code performs UMAP (Uniform Manifold Approximation and Projection) dimensionality reduction on the merged tracks dataframe, focusing on its numeric columns, and visualizes the result. In the provided UMAP code, the parameters `n_neighbors`, `min_dist`, and `n_components` are crucial for determining the structure and appearance of the resulting low-dimensional representation of the data.

<font size = 4>`n_neighbors`: This parameter controls how UMAP balances local versus global structure in the data. It determines the size of the local neighborhood UMAP will look at when learning the manifold structure of the data.
- A smaller value emphasizes the local structure of the data, potentially at the expense of the global structure.
- A larger value allows UMAP to consider more distant neighbors, emphasizing more on the global structure of the data.
- Typically, values in the range of 5 to 50 are chosen, depending on the density and scale of the data.

<font size = 4>`min_dist`: This parameter controls how tightly UMAP is allowed to pack points together. It determines the minimum distance between points in the low-dimensional representation.
- Setting it to a low value will allow points to be packed more closely, potentially revealing clusters in the data.
- A higher value ensures that points are more spread out in the representation.
- Values usually range between 0 and 1.

<font size = 4>`n_dimension`: This parameter determines the number of dimensions in the low-dimensional space that the data will be reduced to.
For visualization purposes, `n_dimension` is typically set to 2 or 3 to obtain 2D or 3D representations, respectively.


In [ ]:
# @title ##Perform UMAP
import umap

n_neighbors = 15  # @param {type: "number"}
min_dist = 0.1  # @param {type: "number"}
n_dimension = 3  # @param {type: "slider", min: 1, max: 3}

# Initialize UMAP object with the specified settings
reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_dimension, random_state=42)
embedding = reducer.fit_transform(selected_df)

# Create dynamic column names based on n_components
column_names = [f'UMAP dimension {i}' for i in range(1, n_dimension + 1)]

# Create a DataFrame with the UMAP results
umap_df = pd.DataFrame(embedding, columns=column_names)

# Concatenate the conditions (if available)
if 'Condition' in merged_tracks_df.columns:
    umap_df = pd.concat([umap_df, merged_tracks_df['Condition'].reset_index(drop=True)], axis=1)

# Visualize the UMAP projection
plt.figure(figsize=(12, 10))

# The plot will adjust automatically based on the n_components
if n_dimension == 2:
    sns.scatterplot(x=column_names[0], y=column_names[1], hue='Condition', data=umap_df, palette='viridis', s=60)
    plt.title('UMAP Projection of the Dataset')
    plt.show()
elif n_dimension == 1:
    sns.stripplot(x=column_names[0], hue='Condition', data=umap_df, palette='viridis', jitter=0.05, size=6)
    plt.title('UMAP Projection of the Dataset')
    plt.show()
else:
    # umap_df should have columns like 'UMAP dimension 1', 'UMAP dimension 2', 'UMAP dimension 3', and 'condition'
    import plotly.express as px
    import pandas as pd
    import numpy as np

    fig = px.scatter_3d(umap_df,
                    x='UMAP dimension 1',
                    y='UMAP dimension 2',
                    z='UMAP dimension 3',
                    color='Condition')

    for trace in fig.data:
      trace.marker.size = 2  # You can set this to any desired value

    fig.show()

### **HDBSCAN**

<font size = 4> The provided code employs HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) to identify clusters within a dataset that has already undergone UMAP dimensionality reduction. HDBSCAN is utilized for its proficiency in determining the optimal number of clusters while managing varied densities within the data.

In the provided HDBSCAN code, the parameters `min_samples`, `min_cluster_size`, and `metric` are crucial for determining the structure and appearance of the resulting clusters in the data.

`min_samples`: This parameter primarily controls the degree to which the algorithm is willing to declare noise. It's the number of samples in a neighborhood for a point to be considered as a core point.
- A smaller value of `min_samples` makes the algorithm more prone to declaring points as part of a cluster, potentially leading to larger clusters and fewer noise points.
- A larger value makes the algorithm more conservative, resulting in more points declared as noise and smaller, more defined clusters.
- The choice of `min_samples` typically depends on the density of the data; denser datasets may require a larger value.

`min_cluster_size`: This parameter determines the smallest size grouping that you wish to consider a cluster.
- A smaller value will allow the formation of smaller clusters, whereas a larger value will prevent small isolated groups of points from being declared as clusters.
- The choice of `min_cluster_size` depends on the scale of the data and the desired level of granularity in the clustering.

`metric`: This parameter is the metric used for distance computation between data points, and it affects the shape of the clusters.
- The `euclidean` metric is a good starting point, and depending on the clustering results and the data type, it might be beneficial to experiment with different metrics.


In [ ]:
# @title ##Run to see more information about the available metrics
print("""
Metric                   Description                                                               Suitable For
-------------------------------------------------------------------------------------------------------------------------------------------------------
Euclidean                Standard distance metric.                                                 Numerical data.
Manhattan                Sum of absolute differences.                                              Numerical/Categorical data.
Chebyshev                Maximum value of absolute differences.                                    Numerical data.
Minkowski                Generalization of Euclidean and Manhattan distance.                       Numerical data.
Bray-Curtis              Dissimilarity between sample sets.                                        Numerical data.
Canberra                 Weighted version of Manhattan distance.                                   Numerical data.
Mahalanobis              Distance between a point and a distribution.                              Numerical data.

""")


In [ ]:
# @title ##Identify clusters using HDBSCAN
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import numpy as np

clustering_data_source = 'umap'  # @param ['umap', 'raw']
min_samples = 15  # @param {type: "number"}
min_cluster_size = 50  # @param {type: "number"}
metric = "euclidean"  # @param ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'braycurtis', 'canberra', 'mahalanobis']

# Apply HDBSCAN
clusterer = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric=metric)  # You may need to tune these parameters

if clustering_data_source == 'umap':
  if n_dimension == 2:
    clusterer.fit(umap_df[['UMAP dimension 1', 'UMAP dimension 2']])  # Use the UMAP results for clustering

  if n_dimension == 3:
    clusterer.fit(umap_df[['UMAP dimension 1', 'UMAP dimension 2', 'UMAP dimension 3']])  # Use the UMAP results for clustering

else:
  clusterer.fit(selected_df.select_dtypes(include=['number']))

# Add the cluster labels to your UMAP DataFrame
umap_df['Cluster'] = clusterer.labels_

# Plotting the results
if n_dimension == 2:

  plt.figure(figsize=(12,10))
  sns.scatterplot(x='UMAP dimension 1', y='UMAP dimension 2', hue='Cluster', palette='set2', data=umap_df, s=60)
  plt.title('Clusters Identified by HDBSCAN')
  plt.show()

if n_dimension == 3:

  fig = px.scatter_3d(umap_df,
                    x='UMAP dimension 1',
                    y='UMAP dimension 2',
                    z='UMAP dimension 3',
                    color='Cluster')

  for trace in fig.data:
    trace.marker.size = 2  # You can set this to any desired value

  fig.show()

In [ ]:
import plotly.express as px  # Importing plotly for 3D plots

# @title ##Identify exemplar cells using HDBSCAN (not available)


# Extracting exemplar points
exemplars = []
for exemplar in clusterer.exemplars_:
    exemplars.extend(exemplar)

# Flatten the exemplars list of lists into a single list
flattened_exemplars = [index for sublist in exemplars for index in sublist]

# Now pass the flattened list to iloc
exemplar_df = umap_df.iloc[flattened_exemplars]

# Plotting clusters and exemplar points
if n_dimension == 1:
    plt.figure(figsize=(12,10))
    sns.stripplot(x='UMAP dimension 1', hue='Cluster', data=umap_df, palette='viridis', jitter=0.05, size=6)
    sns.stripplot(x='UMAP dimension 1', color='red', label='Exemplars', data=exemplar_df, jitter=0.05, size=10, marker='X')
    plt.title('Clusters and Exemplar Cells Identified by HDBSCAN')
    plt.show()

elif n_dimension == 2:
    plt.figure(figsize=(12,10))
    sns.scatterplot(x='UMAP dimension 1', y='UMAP dimension 2', hue='Cluster', palette='viridis', data=umap_df, s=60)
    sns.scatterplot(x='UMAP dimension 1', y='UMAP dimension 2', color='red', label='Exemplars', data=exemplar_df, s=100, marker='X')
    plt.title('Clusters and Exemplar Cells Identified by HDBSCAN')
    plt.show()

elif n_dimension == 3:
    fig = px.scatter_3d(umap_df,
                        x='UMAP dimension 1',
                        y='UMAP dimension 2',
                        z='UMAP dimension 3',
                        color='Cluster',
                        color_discrete_sequence=px.colors.qualitative.Vivid)

    exemplar_fig = px.scatter_3d(exemplar_df,
                                 x='UMAP dimension 1',
                                 y='UMAP dimension 2',
                                 z='UMAP dimension 3',
                                 color='red')

    for trace in fig.data:
        trace.marker.size = 2

    for trace in exemplar_fig.data:
        trace.marker.size = 5
        trace.marker.symbol = 'x'

    fig.add_trace(exemplar_fig.data[0])
    fig.show()


### **Fingerprint**

This section is designed to visualize the distribution of different clusters within each condition in a dataset, showing the 'fingerprint' of each cluster per condition.

In [ ]:
# @title ##Plot the 'fingerprint' of each cluster per condition

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Group by 'Condition' and 'Cluster' and calculate the size of each group
cluster_counts = umap_df.groupby(['Condition', 'Cluster']).size().reset_index(name='counts')

# Calculate the total number of points per condition
total_counts = umap_df.groupby('Condition').size().reset_index(name='total_counts')

# Merge the DataFrames on 'Condition' to calculate percentages
percentage_df = pd.merge(cluster_counts, total_counts, on='Condition')
percentage_df['percentage'] = (percentage_df['counts'] / percentage_df['total_counts']) * 100

# Save the percentage_df DataFrame as a CSV file
percentage_df.to_csv(Results_Folder+'/UMAP_percentage_results.csv', index=False)

# Pivot the percentage_df to have Conditions as index, Clusters as columns, and percentages as values
pivot_df = percentage_df.pivot(index='Condition', columns='Cluster', values='percentage')

# Fill NaN values with 0 if any, as there might be some Condition-Cluster combinations that are not present
pivot_df.fillna(0, inplace=True)

# Initialize PDF
pdf_pages = PdfPages(Results_Folder+'/UMAP_Cluster_Fingerprint_Plot.pdf')

# Plotting
fig, ax = plt.subplots(figsize=(10, 7))
pivot_df.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')
plt.title('Percentage in each cluster per Condition')
plt.ylabel('Percentage')
plt.xlabel('Condition')
plt.xticks(rotation=90)
plt.tight_layout()

# Save the figure to a PDF
pdf_pages.savefig(fig)

# Close the PDF
pdf_pages.close()

# Display the plot
plt.show()



### **4.3: t-SNE and HDBSCAN**

This section performs t-SNE (t-Distributed Stochastic Neighbor Embedding) on your track dataset to reduce its dimensionality and visualize it in 1, 2, or 3 dimensions.

`perplexity`:
Determines the balance between preserving the local and global structure of the data.

*   Typical Values: Between 5 and 50.
*   How to Choose: Higher values consider more neighbors for each point, which could reveal more global structure but might mix different clusters. Lower values focus more on preserving the local structure, potentially leading to more distinct clusters.

`n_iter`:
The number of iterations for the optimization.
  
*  Typical Values: Usually, a minimum of 250.
*  How to Choose: If the results lack stability or the cost function hasn’t converged, consider increasing this value.

`n_dimension`:
The number of dimensions to reduce the data to (1, 2, or 3 for this code).

*   How to Choose: Typically 2 or 3 for visualization purposes. Choose 1 for a linear representation.



In [ ]:
# @title ##Perform t-SNE

import pandas as pd
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

perplexity = 30  # @param {type: "number"}
n_iter = 300  # @param {type: "number"}
n_dimension = 3  # @param {type: "slider", min: 1, max: 3}

scaled_df = StandardScaler().fit_transform(selected_df)  # Scaling the numeric features

tsne = TSNE(n_components=n_dimension, perplexity=perplexity, n_iter=n_iter, random_state=42)
tsne_results = tsne.fit_transform(scaled_df)

# Adjust the DataFrame creation to handle 1, 2, or 3 dimensions
dimension_columns = [f'Dimension {i}' for i in range(1, n_dimension + 1)]
tsne_df = pd.DataFrame(data=tsne_results, columns=dimension_columns)
tsne_df['Condition'] = merged_tracks_df['Condition'].reset_index(drop=True)  # Reset index to avoid misalignment

# Plotting the results
if n_dimension == 1:
    plt.figure(figsize=(10,8))
    sns.stripplot(x='Dimension 1', hue='Condition', data=tsne_df, palette='viridis', jitter=0.05, size=6)
    plt.title('t-SNE Plot')
    plt.show()

elif n_dimension == 2:
    plt.figure(figsize=(10,8))
    sns.scatterplot(
        x='Dimension 1', y='Dimension 2',
        hue='Condition',
        palette=sns.color_palette("hsv", len(tsne_df['Condition'].unique())),
        data=tsne_df,
        legend="full",
        alpha=0.9
    )
    plt.title('t-SNE Plot')
    plt.show()

elif n_dimension == 3:
    import plotly.express as px  # Ensure plotly is imported

    fig = px.scatter_3d(tsne_df,
                        x='Dimension 1',
                        y='Dimension 2',
                        z='Dimension 3',
                        color='Condition',  # Use 'Condition' as color
                        color_discrete_sequence=px.colors.qualitative.Vivid)  # Use a color sequence suitable for categorical data

    for trace in fig.data:
        trace.marker.size = 2  # You can set this to any desired value

    fig.show()

In [ ]:
# @title ##Identify clusters using HDBSCAN

import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import numpy as np

clustering_data_source = 'tsne'  # @param ['tsne', 'raw']
min_samples = 15  # @param {type: "number"}
min_cluster_size = 50  # @param {type: "number"}
metric = "euclidean"  # @param ['euclidean', 'manhattan', 'chebyshev', 'minkowski', 'braycurtis', 'canberra', 'mahalanobis']

# Apply HDBSCAN
clusterer = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric=metric)

if clustering_data_source == 'tsne':
  clusterer.fit(tsne_results)
else:
  clusterer.fit(selected_df.select_dtypes(include=['number']))

# Add cluster labels to the DataFrame
tsne_df['Cluster'] = clusterer.labels_

# Plotting the results
if n_dimension == 2:
  plt.figure(figsize=(12,10))
  sns.scatterplot(x='Dimension 1', y='Dimension 2', hue='Cluster', palette='set2', data=tsne_df, s=60)
  plt.title('Clusters Identified by HDBSCAN')
  plt.show()

elif n_dimension == 3:
  import plotly.express as px  # Ensure plotly is imported
  fig = px.scatter_3d(tsne_df,
                      x='Dimension 1',
                      y='Dimension 2',
                      z='Dimension 3',
                      color='Cluster')
  for trace in fig.data:
    trace.marker.size = 2
  fig.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# @title ##Plot the 'fingerprint' of each cluster per condition



# Group by 'Condition' and 'Cluster' and calculate the size of each group
cluster_counts = tsne_df.groupby(['Condition', 'Cluster']).size().reset_index(name='counts')

# Calculate the total number of points per condition
total_counts = tsne_df.groupby('Condition').size().reset_index(name='total_counts')

# Merge the DataFrames on 'Condition' to calculate percentages
percentage_df = pd.merge(cluster_counts, total_counts, on='Condition')
percentage_df['percentage'] = (percentage_df['counts'] / percentage_df['total_counts']) * 100

# Save the percentage_df DataFrame as a CSV file
percentage_df.to_csv(Results_Folder+'/tsne_percentage_results.csv', index=False)

# Pivot the percentage_df to have Conditions as index, Clusters as columns, and percentages as values
pivot_df = percentage_df.pivot(index='Condition', columns='Cluster', values='percentage')

# Fill NaN values with 0 if any, as there might be some Condition-Cluster combinations that are not present
pivot_df.fillna(0, inplace=True)

# Initialize PDF
pdf_pages = PdfPages(Results_Folder+'/tsne_Cluster_Fingerprint_Plot.pdf')

# Plotting
fig, ax = plt.subplots(figsize=(10, 7))
pivot_df.plot(kind='bar', stacked=True, ax=ax, colormap='viridis')
plt.title('Percentage in each cluster per Condition')
plt.ylabel('Percentage')
plt.xlabel('Condition')
plt.xticks(rotation=90)
plt.tight_layout()

# Save the figure to a PDF
pdf_pages.savefig(fig)

# Close the PDF
pdf_pages.close()

# Display the plot
plt.show()
